SUPERSELECTOR

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from collections import Counter
from sklearn.feature_selection import SelectKBest, f_classif, SelectFromModel, RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from mlxtend.feature_selection import SequentialFeatureSelector
from sklearn.model_selection import train_test_split, GridSearchCV
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

In [18]:
def super_selector(dataset, target_col="", selectores={}, hard_voting=[]):
    """
    Selecciona características de un dataframe de features según diferentes métodos de selección.

    Args:
        dataset (pandas.DataFrame): El dataframe de características.
        target_col (str): El nombre de la columna objetivo.
        selectores (dict): Un diccionario con métodos de selección como claves y sus parámetros como valores.
        hard_voting (list): Una lista de características para realizar hard voting.

    Returns:
        dict: Un diccionario que contiene las características seleccionadas por cada método de selección
              y el resultado del hard voting si se especifica.

    Ejemplo:
        selectores = {
            "KBest": 5,
            "FromModel": [RandomForestClassifier(), 5],
            "RFE": [LogisticRegression(), 5, 1]
        }
        result = super_selector(train_set_titanic, target_col="Survived", selectores=selectores, hard_voting=["Pclass", "who", "embarked_S", "fare", "age"])
        print(result)
    """

    result = {}  # Se inicializa un diccionario para almacenar el resultado de la selección de características.

    # Verificar si target_col es válido
    if target_col and target_col in dataset.columns:  # Verifica si se proporcionó un nombre de columna objetivo y si existe en el dataframe.
        target = dataset[target_col]  # Guarda la columna objetivo en 'target'.
        features = dataset.drop(columns=[target_col])  # Elimina la columna objetivo del dataframe de características y guarda el resto en 'features'.
    else:
        target = None  # Si no se proporciona una columna objetivo válida, se establece como 'None'.
        features = dataset.copy()  # Si no se proporciona una columna objetivo válida, se copian todas las características del dataframe.

    # Seleccionar características basadas en selectores
    for selector, params in selectores.items():  # Itera sobre cada método de selección y sus parámetros.
        if selector == "KBest":  # Si el método de selección es 'KBest' (Selección de las mejores características basadas en pruebas univariadas).
            k_best = SelectKBest(score_func=f_classif, k=params)  # Inicializa el selector de las mejores características.
            k_best.fit(features, target)  # Ajusta el selector a las características y el objetivo.
            selected_features = features.columns[k_best.get_support()]  # Obtiene las características seleccionadas.
            result[selector] = selected_features.tolist()  # Almacena las características seleccionadas en el resultado.

        elif selector == "FromModel":  # Si el método de selección es 'FromModel' (Selección de características basadas en un modelo externo).
            model, threshold = params  # Obtiene el modelo y el umbral de los parámetros.
            if isinstance(threshold, int):  # Comprueba si el umbral es un número entero.
                max_features = threshold  # Si es un número entero, se establece como el número máximo de características.
                threshold = -np.inf  # El umbral se establece en infinito negativo.
            else:
                max_features = None  # Si no es un número entero, el número máximo de características no se limita.
            selector_model = SelectFromModel(model, threshold=threshold, max_features=max_features)  # Inicializa el selector basado en el modelo.
            selector_model.fit(features, target)  # Ajusta el selector a las características y el objetivo.
            selected_features = features.columns[selector_model.get_support()]  # Obtiene las características seleccionadas.
            result[selector] = selected_features.tolist()  # Almacena las características seleccionadas en el resultado.

        elif selector == "RFE":  # Si el método de selección es 'RFE' (Eliminación recursiva de características).
            model, n_features, step = params  # Obtiene el modelo, el número de características y el paso de los parámetros.
            rfe = RFE(estimator=model, n_features_to_select=n_features, step=step)  # Inicializa el selector de RFE.
            rfe.fit(features, target)  # Ajusta el selector a las características y el objetivo.
            selected_features = features.columns[rfe.support_]  # Obtiene las características seleccionadas.
            result[selector] = selected_features.tolist()  # Almacena las características seleccionadas en el resultado.

        elif selector == "SFS":  # Si el método de selección es 'SFS' (Selección secuencial hacia adelante).
            model, n_features = params  # Obtiene el modelo y el número de características de los parámetros.
            sfs = SequentialFeatureSelector(model, k_features=n_features, forward=True, floating=False, scoring='accuracy', cv=StratifiedKFold(5))  # Inicializa el selector SFS.
            sfs.fit(features, target)  # Ajusta el selector a las características y el objetivo.
            selected_features = features.columns[list(sfs.k_feature_idx_)]  # Obtiene las características seleccionadas.
            result[selector] = selected_features.tolist()  # Almacena las características seleccionadas en el resultado.

    # Realizar hard voting
    
    all_features = [result[key] for key in result]  # Combinar todas las listas seleccionadas
    all_features.extend(hard_voting)
    if all_features:
        voting_counts = Counter([feature for sublist in all_features for feature in sublist if feature in features.columns])
        sorted_voting = sorted(voting_counts.items(), key=lambda x: x[1], reverse=True)
        hard_voting_result = [feature[0] for feature in sorted_voting[:len(hard_voting)]]
        result['hard_voting'] = hard_voting_result
    elif len(result) == 1:
        result['hard_voting'] = result[next(iter(result))]

    return result

    return result  # Devuelve el resultado que contiene las características seleccionadas por cada método de selección y el resultado del hard voting si se especifica.

In [19]:
diamonds = pd.read_csv("diamonds_preparado.csv", sep = "|")

In [20]:
diamonds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 23 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   carat         53940 non-null  float64
 1   cut           53940 non-null  object 
 2   depth         53940 non-null  float64
 3   table         53940 non-null  float64
 4   price         53940 non-null  int64  
 5   x             53940 non-null  float64
 6   y             53940 non-null  float64
 7   z             53940 non-null  float64
 8   color_D       53940 non-null  int64  
 9   color_E       53940 non-null  int64  
 10  color_F       53940 non-null  int64  
 11  color_G       53940 non-null  int64  
 12  color_H       53940 non-null  int64  
 13  color_I       53940 non-null  int64  
 14  color_J       53940 non-null  int64  
 15  clarity_IF    53940 non-null  int64  
 16  clarity_VVS1  53940 non-null  int64  
 17  clarity_VVS2  53940 non-null  int64  
 18  clarity_VS1   53940 non-nu

In [21]:
# Crear un diccionario de mapeo de etiquetas a valores numéricos
mapping = {'Ideal': 5, 'Premium': 4, 'Very Good': 3, 'Good': 2, 'Fair': 1}

# Aplicar el mapeo al DataFrame
diamonds['cut'] = diamonds['cut'].map(mapping)

In [22]:
diamonds

,carat,cut,depth,table,price,x,y,z,color_D,color_E,...,color_I,color_J,clarity_IF,clarity_VVS1,clarity_VVS2,clarity_VS1,clarity_VS2,clarity_SI1,clarity_SI2,clarity_I1
0,0.23,5,61.5,55.0,326,3.95,3.98,2.43,0,1,...,0,0,0,0,0,0,0,0,1,0
1,0.21,4,59.8,61.0,326,3.89,3.84,2.31,0,1,...,0,0,0,0,0,0,0,1,0,0
2,0.23,2,56.9,65.0,327,4.05,4.07,2.31,0,1,...,0,0,0,0,0,1,0,0,0,0
3,0.29,4,62.4,58.0,334,4.20,4.23,2.63,0,0,...,1,0,0,0,0,0,1,0,0,0
4,0.31,2,63.3,58.0,335,4.34,4.35,2.75,0,0,...,0,1,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53935,0.72,5,60.8,57.0,2757,5.75,5.76,3.50,1,0,...,0,0,0,0,0,0,0,1,0,0
53936,0.72,2,63.1,55.0,2757,5.69,5.75,3.61,1,0,...,0,0,0,0,0,0,0,1,0,0
53937,0.70,3,62.8,60.0,2757,5.66,5.68,3.56,1,0,...,0,0,0,0,0,0,0,1,0,0
53938,0.86,4,61.0,58.0,2757,6.15,6.12,3.74,0,0,...,0,0,0,0,0,0,0,0,1,0


In [23]:
from sklearn.preprocessing import StandardScaler
diamonds = pd.DataFrame(StandardScaler.fit_transform(diamonds), columns=diamonds.columns)


TypeError: TransformerMixin.fit_transform() missing 1 required positional argument: 'X'

In [11]:
train_set_diamonds, test_set_diamonds = train_test_split(diamonds, test_size = 0.2, random_state = 42)


In [12]:
# Ejemplo de uso
selectores = {
    "KBest": 5,
    "FromModel": [RandomForestClassifier(), 5],
    "RFE": [LogisticRegression(), 5, 1],
    "SFS": [RandomForestClassifier(), 5] 
}
result = super_selector(train_set_diamonds, target_col="cut", selectores=selectores, hard_voting=["depth","price","table","y","clarity_VVS1"])
result

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://sciki

TypeError: 'NoneType' object is not iterable

Si hard_voting era lista vacia:

In [ ]:
# Ejemplo de uso
selectores = {
    "KBest": 5,
    "FromModel": [RandomForestClassifier(), 5],
    "RFE": [LogisticRegression(), 5, 1],
    "SFS": [RandomForestClassifier(), 5] 
}
result = super_selector(train_set_diamonds, target_col="survived", selectores=selectores, hard_voting=[])
result

{'KBest': ['pclass', 'fare', 'who', 'alone', 'embarked_C'],
 'FromModel': ['pclass', 'age', 'sibsp', 'fare', 'who'],
 'RFE': ['pclass', 'sibsp', 'who', 'alone', 'embarked_C'],
 'SFS': ['pclass', 'sibsp', 'who', 'alone', 'embarked_Q']}